In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
import pyspark

In [4]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType
import functools
import pandas as pd
#pd.options.display.max_colwidth = None

In [5]:
book = 'hdfs://master:9000/user/practice/book_data.csv'
isbn_summary = 'hdfs://master:9000/user/practice/sbn_review.csv'

In [6]:
sc1 = SparkSession.builder.appName('book_data').getOrCreate()
sc2 = SparkSession.builder.appName('isbn_review').getOrCreate()

In [7]:
books = sc1.read.csv(book, header=True, inferSchema=True)
books.printSchema()

root
 |-- main_category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- book_id: integer (nullable = true)
 |-- book_title: string (nullable = true)
 |-- book_author: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- publish_date: string (nullable = true)
 |-- aladin_price: integer (nullable = true)
 |-- aladin_star: double (nullable = true)
 |-- aladin_review: integer (nullable = true)
 |-- kyobo_price: integer (nullable = true)
 |-- kyobo_star: double (nullable = true)
 |-- kyobo_review: integer (nullable = true)
 |-- yes24_price: integer (nullable = true)
 |-- yes24_star: double (nullable = true)
 |-- yes24_review: integer (nullable = true)
 |-- interpark_price: integer (nullable = true)
 |-- interpark_star: double (nullable = true)
 |-- interpark_review: integer (nullable = true)



In [ ]:
books.limit(10).toPandas()

In [8]:
isbns = sc2.read.option('multiline', True).csv(isbn_summary, header=True, inferSchema=True)
isbns.printSchema()
isbns.columns

root
 |-- book_id: integer (nullable = true)
 |-- book_isbn: string (nullable = true)
: string (nullable = true)



['book_id', 'book_isbn', 'book_trailer\r']

In [9]:
isbns = isbns.select(col('book_id'), col('book_isbn'), col('book_trailer\r').alias('book_trailer'))

In [ ]:
# isbns = isbns.distinct()
isbns.toPandas()

In [10]:
category = books.groupBy('main_category').agg(count('main_category'))
category.show(25)

+--------------+--------------------+
| main_category|count(main_category)|
+--------------+--------------------+
|   국어/외국어|                 409|
|      해외도서|                 350|
|     취미/레저|                 797|
|          유아|                 456|
|     여행/지도|                 240|
|          건강|                 422|
|   학습/참고서|                 255|
|          잡지|                  60|
|          인문|                 656|
|   취업/수험서|                 203|
|     경제/경영|                 561|
|          만화|                 852|
|        청소년|                 591|
|     시/에세이|                 787|
|          소설|                 390|
|     컴퓨터/IT|                 396|
|          사회|                 421|
|        어린이|                 455|
| 예술/대중문화|                 632|
|     역사/문화|                 137|
|          종교|                 357|
|가정/생활/요리|                 412|
|     과학/공학|                 835|
|      자기계발|                 455|
|          사전|                 127|
+--------------+------

# 모든 플랫폼 중에 최저가격 확인

In [11]:
books = books.withColumn("min price", least('aladin_price', 'kyobo_price', 'yes24_price', 'interpark_price'))
books.limit(10).toPandas()

,main_category,sub_category,book_id,book_title,book_author,publisher,publish_date,aladin_price,aladin_star,aladin_review,kyobo_price,kyobo_star,kyobo_review,yes24_price,yes24_star,yes24_review,interpark_price,interpark_star,interpark_review,min price
0,가정/생활/요리,요리,21354394,에듀윌 제과 제빵기능사 필기끝장,오명석 장다예 박진홍 외,에듀윌,2022.01.09,18000,10.0,7,18000,9.4,3,20000,10.0,6,18000,10.0,5,18000
1,가정/생활/요리,집/살림,14518950,노후주택 리모델링,전원속의 내집 편집부,주택문화사,2019.01.24,24300,10.0,1,24300,9.8,0,27000,8.8,5,24300,10.0,1,24300
2,가정/생활/요리,집/살림,16853326,실내에서 이끼키우기,이선희 박웅택 정혜원 이은정,플로라,2020.10.20,14400,10.0,1,14400,8.3,0,16000,10.0,1,14400,0.0,0,14400
3,가정/생활/요리,집/살림,6860734,내 손으로 직접하는 나무 가지치기,편집부,그린홈,2012.03.15,17100,7.5,4,17100,9.7,1,19000,9.2,6,17100,8.0,1,17100
4,가정/생활/요리,집/살림,12863883,시공사도 건축가도 안 알려주는 건축주만이 알려줄 수 있는 집짓기 진실,손창완,주택문화사,2017.12.06,20700,10.0,4,20700,9.2,0,23000,8.8,4,20700,0.0,0,20700
5,가정/생활/요리,집/살림,16290376,선인장도 말려 죽이는 그대에게,송한나,책밥,2020.03.23,16920,9.2,12,16920,9.2,6,18800,9.2,16,16920,9.1,7,16920
6,가정/생활/요리,집/살림,22202689,방구석 가드닝,앤절라 S. 저드,문학수첩,2022.03.25,12600,10.0,4,12600,10.0,1,14000,10.0,5,12600,0.0,0,12600
7,가정/생활/요리,집/살림,8843474,인테리어 원 북,윤소연,디자인하우스,2015.04.05,14400,9.0,36,14400,9.2,20,16000,8.9,25,14400,9.9,15,14400
8,가정/생활/요리,집/살림,7306017,텃밭정원 가이드북,오도,그물코,2013.09.10,13500,9.5,4,13500,9.0,0,15000,8.8,4,13500,9.3,1,13500
9,가정/생활/요리,집/살림,7361595,정원의 발견,오경아,궁리,2013.11.15,20700,8.4,11,20700,10.0,6,23000,9.4,14,20700,9.0,2,20700


# 모든 플랫폼에서 평점평균 확인

In [12]:
books = books.withColumn("avg stars", (col('aladin_star')+col('kyobo_star')+col('yes24_star')+col('interpark_star'))/4)
books.limit(10).toPandas()

,main_category,sub_category,book_id,book_title,book_author,publisher,publish_date,aladin_price,aladin_star,aladin_review,...,kyobo_star,kyobo_review,yes24_price,yes24_star,yes24_review,interpark_price,interpark_star,interpark_review,min price,avg stars
0,가정/생활/요리,요리,21354394,에듀윌 제과 제빵기능사 필기끝장,오명석 장다예 박진홍 외,에듀윌,2022.01.09,18000,10.0,7,...,9.4,3,20000,10.0,6,18000,10.0,5,18000,9.850
1,가정/생활/요리,집/살림,14518950,노후주택 리모델링,전원속의 내집 편집부,주택문화사,2019.01.24,24300,10.0,1,...,9.8,0,27000,8.8,5,24300,10.0,1,24300,9.650
2,가정/생활/요리,집/살림,16853326,실내에서 이끼키우기,이선희 박웅택 정혜원 이은정,플로라,2020.10.20,14400,10.0,1,...,8.3,0,16000,10.0,1,14400,0.0,0,14400,7.075
3,가정/생활/요리,집/살림,6860734,내 손으로 직접하는 나무 가지치기,편집부,그린홈,2012.03.15,17100,7.5,4,...,9.7,1,19000,9.2,6,17100,8.0,1,17100,8.600
4,가정/생활/요리,집/살림,12863883,시공사도 건축가도 안 알려주는 건축주만이 알려줄 수 있는 집짓기 진실,손창완,주택문화사,2017.12.06,20700,10.0,4,...,9.2,0,23000,8.8,4,20700,0.0,0,20700,7.000
5,가정/생활/요리,집/살림,16290376,선인장도 말려 죽이는 그대에게,송한나,책밥,2020.03.23,16920,9.2,12,...,9.2,6,18800,9.2,16,16920,9.1,7,16920,9.175
6,가정/생활/요리,집/살림,22202689,방구석 가드닝,앤절라 S. 저드,문학수첩,2022.03.25,12600,10.0,4,...,10.0,1,14000,10.0,5,12600,0.0,0,12600,7.500
7,가정/생활/요리,집/살림,8843474,인테리어 원 북,윤소연,디자인하우스,2015.04.05,14400,9.0,36,...,9.2,20,16000,8.9,25,14400,9.9,15,14400,9.250
8,가정/생활/요리,집/살림,7306017,텃밭정원 가이드북,오도,그물코,2013.09.10,13500,9.5,4,...,9.0,0,15000,8.8,4,13500,9.3,1,13500,9.150
9,가정/생활/요리,집/살림,7361595,정원의 발견,오경아,궁리,2013.11.15,20700,8.4,11,...,10.0,6,23000,9.4,14,20700,9.0,2,20700,9.200


# 모든 플랫폼에서 리뷰 총 개수

In [13]:
books = books.withColumn("num of reviews", col('aladin_review')+col('kyobo_review')+col('yes24_review')+col('interpark_review'))
books.limit(10).toPandas()

,main_category,sub_category,book_id,book_title,book_author,publisher,publish_date,aladin_price,aladin_star,aladin_review,...,kyobo_review,yes24_price,yes24_star,yes24_review,interpark_price,interpark_star,interpark_review,min price,avg stars,num of reviews
0,가정/생활/요리,요리,21354394,에듀윌 제과 제빵기능사 필기끝장,오명석 장다예 박진홍 외,에듀윌,2022.01.09,18000,10.0,7,...,3,20000,10.0,6,18000,10.0,5,18000,9.850,21
1,가정/생활/요리,집/살림,14518950,노후주택 리모델링,전원속의 내집 편집부,주택문화사,2019.01.24,24300,10.0,1,...,0,27000,8.8,5,24300,10.0,1,24300,9.650,7
2,가정/생활/요리,집/살림,16853326,실내에서 이끼키우기,이선희 박웅택 정혜원 이은정,플로라,2020.10.20,14400,10.0,1,...,0,16000,10.0,1,14400,0.0,0,14400,7.075,2
3,가정/생활/요리,집/살림,6860734,내 손으로 직접하는 나무 가지치기,편집부,그린홈,2012.03.15,17100,7.5,4,...,1,19000,9.2,6,17100,8.0,1,17100,8.600,12
4,가정/생활/요리,집/살림,12863883,시공사도 건축가도 안 알려주는 건축주만이 알려줄 수 있는 집짓기 진실,손창완,주택문화사,2017.12.06,20700,10.0,4,...,0,23000,8.8,4,20700,0.0,0,20700,7.000,8
5,가정/생활/요리,집/살림,16290376,선인장도 말려 죽이는 그대에게,송한나,책밥,2020.03.23,16920,9.2,12,...,6,18800,9.2,16,16920,9.1,7,16920,9.175,41
6,가정/생활/요리,집/살림,22202689,방구석 가드닝,앤절라 S. 저드,문학수첩,2022.03.25,12600,10.0,4,...,1,14000,10.0,5,12600,0.0,0,12600,7.500,10
7,가정/생활/요리,집/살림,8843474,인테리어 원 북,윤소연,디자인하우스,2015.04.05,14400,9.0,36,...,20,16000,8.9,25,14400,9.9,15,14400,9.250,96
8,가정/생활/요리,집/살림,7306017,텃밭정원 가이드북,오도,그물코,2013.09.10,13500,9.5,4,...,0,15000,8.8,4,13500,9.3,1,13500,9.150,9
9,가정/생활/요리,집/살림,7361595,정원의 발견,오경아,궁리,2013.11.15,20700,8.4,11,...,6,23000,9.4,14,20700,9.0,2,20700,9.200,33


# trailer & isbn 추가

In [14]:
books = books.join(isbns, on = ['book_id'], how='left_outer')

In [16]:
books.orderBy(col('main_category').asc(), col('sub_category').asc()).limit(10).toPandas()

,book_id,main_category,sub_category,book_title,book_author,publisher,publish_date,aladin_price,aladin_star,aladin_review,...,yes24_star,yes24_review,interpark_price,interpark_star,interpark_review,min price,avg stars,num of reviews,book_isbn,book_trailer
0,10652913,가정/생활/요리,결혼/가족,우리 부부 어디서 잘못된 걸까?,이병준,영진닷컴,2016.06.03,11700,0.0,0,...,10.0,1,11700,0.0,0,11700,5.000,1,None,None
1,7170355,가정/생활/요리,결혼/가족,프랑스 아이처럼,파멜라 드러커맨,북하이브,2013.03.20,13500,8.9,100,...,8.6,88,13500,8.6,25,13500,8.925,216,9788928615360,"나와 아이가 조금 더 행복해지는 프랑스식 육아법!\r\n\r\n아이, 엄마, 가족이..."
2,16894194,가정/생활/요리,결혼/가족,별난 내과의사가 알려주는 정력을 키우는 방법,조왕기,린쓰,2020.10.30,10800,8.9,9,...,8.7,8,10800,8.0,2,10800,8.900,22,9791196054984,정력이라고 하면 좀 낯뜨거운 거 같다고?\r\n나도 몰랐던 내 몸의 기능이 되살아난...
3,7170355,가정/생활/요리,결혼/가족,프랑스 아이처럼,파멜라 드러커맨,북하이브,2013.03.20,13500,8.9,100,...,8.6,88,13500,8.6,25,13500,8.925,216,9788928615360,"나와 아이가 조금 더 행복해지는 프랑스식 육아법!\r\n\r\n아이, 엄마, 가족이..."
4,7170355,가정/생활/요리,결혼/가족,프랑스 아이처럼,파멜라 드러커맨,북하이브,2013.03.20,13500,8.9,100,...,8.6,88,13500,8.6,25,13500,8.925,216,9788928615360,"나와 아이가 조금 더 행복해지는 프랑스식 육아법!\r\n\r\n아이, 엄마, 가족이..."
5,118719,가정/생활/요리,결혼/가족,그 남자가 원하는 여자 그 여자가 원하는 남자,김성묵,김영사,2003.01.10,8910,7.8,11,...,8.2,11,8910,8.5,7,8910,8.325,30,9788934912088,"교회 사역장로 및 한국가정사역학회 부회장, 두란노아버지학교운동본부장으로 활동하며 가..."
6,18059178,가정/생활/요리,결혼/가족,우리가 사랑할 때 물어야 할 여덟 가지,존 가트맨 줄리 슈워츠 가트맨 더글러스 에이브럼스 레이철 칼턴 에이브럼스,해냄출판사,2021.03.05,14850,9.7,26,...,9.6,24,14850,9.1,7,14850,9.600,71,9788965743408,아마존 결혼 · 가정 분야 1위!\r\n\r\n2019 노틸러스 북어워드 ‘인간관계...
7,7916309,가정/생활/요리,결혼/가족,회복탄력성,최성애,해냄출판사,2014.07.28,14850,9.0,30,...,9.3,49,14850,9.3,34,14850,9.400,137,9788965744528,누군가를 돌보고 가르치는 사람들에게 전하는 스트레스 조절과 관계의 기술!\r\n\r...
8,7170355,가정/생활/요리,결혼/가족,프랑스 아이처럼,파멜라 드러커맨,북하이브,2013.03.20,13500,8.9,100,...,8.6,88,13500,8.6,25,13500,8.925,216,9788928615360,"나와 아이가 조금 더 행복해지는 프랑스식 육아법!\r\n\r\n아이, 엄마, 가족이..."
9,11823851,가정/생활/요리,결혼/가족,행복한 결혼을 위한 7원칙,존 가트맨 낸 실버,문학사상,2017.03.22,13500,0.0,0,...,10.0,1,13500,8.0,1,13500,7.000,2,9788970129396,존 가트맨 박사는 수십 년 동안 수많은 부부들을 연구하는 과정에서 행복한 부부와 이...


# 중복제거

In [17]:
books = books.dropDuplicates(['book_id'])

In [18]:
books.limit(10).toPandas()

,book_id,main_category,sub_category,book_title,book_author,publisher,publish_date,aladin_price,aladin_star,aladin_review,...,yes24_star,yes24_review,interpark_price,interpark_star,interpark_review,min price,avg stars,num of reviews,book_isbn,book_trailer
0,182678,어린이,초등1~2학년,종이 봉지 공주,버트 먼치,비룡소,2008.07.18,9900,9.2,131,...,9.0,76,9900,9.2,56,9900,9.250,264,9788949110479,왕자를 구한 공주의 운명은?\r\n\r\n안데르센의 동화 「돼지치기 소년」을 페미니...
1,2084345,역사/문화,역사/문화일반,총 균 쇠,재레드 다이아몬드,문학사상사,2005.12.19,25200,9.0,301,...,9.2,241,25200,9.2,134,25200,9.125,677,9788970127248,왜 어떤 민족들은 다른 민족들의 정복과 지배의 대상으로 전락하고 말았는가. 왜 원주...
2,6050510,어린이,초등5~6학년,불량한 자전거 여행,남중,창비,2009.07.28,9720,9.6,92,...,9.5,79,9720,9.8,22,9720,9.600,196,9788936442507,"뜨거운 여름 1,100킬로 자전거 여행 한 소년이 그 길에서 준비한 '불량한 출발'..."
3,6055863,소설,고전/문학,1Q84,무라카미 하루키,문학동네,2009.09.08,13320,8.6,327,...,8.7,130,13320,9.2,203,13320,9.050,692,9788954608657,당신의 하늘에는 몇 개의 달이 떠 있습니까?\r\n\r\n무라카미 하루키가 <어둠의...
4,6348922,인문,종교학,종교의 미래,하비 콕스,문예출판사,2010.08.25,15300,9.1,11,...,8.4,14,15300,8.9,10,15300,8.800,42,9788931006810,세계적인 종교학자 하비콕스가 내놓은 그리스도교의 미래와 전망\r\n\r\n예수의 시...
5,6597874,취미/레저,옷만들기,초보자를 위한 친환경 가구 만들기,우상연,북하우스엔,2011.03.19,18000,8.2,20,...,8.6,9,18000,9.3,10,18000,8.600,42,9788956055152,친절한 가구 제작 안내서\r\n\r\n공구 사용법부터 가구 제작법까지 우리집 목공 ...
6,6793587,해외도서,취미/여행,Pantone Postcard Box: 100 Postcards (Novelty),Pantone INC,Chronicle Books (CA),2011.06.22,16500,0.0,0,...,9.2,12,17500,0.0,0,16500,4.175,13,9780811877541,WITH A PALETTE DRAWN FROM THE SYSTEMS OF PANTO...
7,6822225,청소년,청소년 문학,방관자,제임스 프렐러,미래인,2012.03.05,8550,9.2,29,...,8.8,43,8550,9.4,27,8550,9.300,100,9788983946966,방관자는 다음 피해자인가?\r\n\r\n가해자와 피해자 사이에서 갈등하는 방관자의 ...
8,7820011,예술/대중문화,사진,가문비나무의 노래,마틴 슐레스케,니케북스,2014.07.10,13500,10.0,5,...,9.3,17,13500,9.4,6,13500,9.650,28,9788994361147,독일의 바이올린 장인 마틴 슐레스케가 작업장에서 길어 올린 365개의 맑은 생각과 ...
9,8882146,역사/문화,역사/문화일반,역사란 무엇인가,에드워드 카,까치,2015.03.16,10800,9.2,68,...,8.7,35,10800,8.8,2,10800,9.050,106,9788972915812,역사란 ‘과거와 현재의 대화’ 또는 ‘과거의 사실과 현재의 역사가의 대화’라는 것은...


# 플랫폼별 가격, 별점, 리뷰 평균

In [19]:
books_platform = books.agg(avg("aladin_price").alias('avg_aladin_price'), avg("aladin_star").alias('avg_aladin_star'), 
             avg("aladin_review").alias('avg_aladin_review'), avg("kyobo_price").alias('avg_kyobo_price'),
             avg("kyobo_star").alias('avg_kyobo_star'), avg("kyobo_review").alias('avg_kyobo_review'),
             avg("yes24_price").alias('avg_yes24_price'), avg("yes24_star").alias('avg_yes24_star'),
             avg("yes24_review").alias('avg_yes24_review'), avg("interpark_price").alias('avg_interpark_price'),
             avg("interpark_star").alias('avg_interpark_star'), avg("interpark_review").alias('avg_interpark_review'))
books_platform.limit(10).toPandas()

,avg_aladin_price,avg_aladin_star,avg_aladin_review,avg_kyobo_price,avg_kyobo_star,avg_kyobo_review,avg_yes24_price,avg_yes24_star,avg_yes24_review,avg_interpark_price,avg_interpark_star,avg_interpark_review
0,15371.036493,8.269974,27.032606,15357.971281,9.533902,4.541892,17011.343122,9.358076,27.013064,15357.971281,6.454394,7.54146


In [ ]:
books_platform.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("/home/hadoop/venv/books_platform.csv")

# main category별 별점, 가격, 리뷰개수 평균

### 별점 평균

In [28]:
books_main_star = books.groupby(books['main_category']).agg({'aladin_star':'mean', 'kyobo_star': 'mean', 'yes24_star': 'mean', 
                                                        'interpark_star': 'mean'})

books_main_star = books_main_star.withColumn("avg stars", (col('avg(aladin_star)') + col('avg(kyobo_star)') + col('avg(yes24_star)') + col('avg(interpark_star)'))/4)
books_main_star.orderBy(col('avg stars').desc()).toPandas()                    

,main_category,avg(aladin_star),avg(kyobo_star),avg(yes24_star),avg(interpark_star),avg stars
0,어린이,9.186822,9.719380,9.603618,8.128682,9.159625
1,소설,8.815077,9.524000,9.174154,8.222462,8.933923
2,자기계발,8.783333,9.452339,9.227485,8.135673,8.899708
3,경제/경영,8.757552,9.422396,9.339583,8.046615,8.891536
4,유아,8.601553,9.634783,9.535404,7.593789,8.841382
5,가정/생활/요리,8.987841,9.511166,9.501489,7.141191,8.785422
6,인문,8.673541,9.486770,9.209533,7.416148,8.696498
7,청소년,8.429797,9.574041,9.385553,7.370429,8.689955
8,시/에세이,8.909859,9.483380,9.250986,6.959296,8.650880
9,건강,8.457143,9.423292,9.311180,7.265528,8.614286


### 가격 평균

In [29]:
books_main_price = books.groupby(books['main_category']).agg({'aladin_price':'mean', 
                                                              'kyobo_price': 'mean', 
                                                              'yes24_price': 'mean', 
                                                              'interpark_price': 'mean'})

books_main_price = books_main_price.withColumn("avg price", 
                                               (col('avg(aladin_price)') + col('avg(kyobo_price)') 
                                                + col('avg(yes24_price)') + col('avg(interpark_price)'))/4)
books_main_price.orderBy(col('avg price').desc()).toPandas() 

,main_category,avg(aladin_price),avg(kyobo_price),avg(interpark_price),avg(yes24_price),avg price
0,사전,32003.534483,31939.310345,31939.310345,34956.034483,32709.547414
1,취업/수험서,22731.875000,22623.812500,22623.812500,25118.125000,23274.406250
2,컴퓨터/IT,21640.314465,21625.314465,21625.314465,23826.729560,22179.418239
3,과학/공학,21199.585714,21093.657143,21093.657143,22788.714286,21543.903571
4,역사/문화,19174.646465,19112.727273,19112.727273,21208.080808,19652.045455
5,사회,17582.331606,17515.595855,17515.595855,19253.886010,17966.852332
6,잡지,17356.785714,17246.071429,17246.071429,18857.142857,17676.517857
7,여행/지도,16841.044776,16836.567164,16836.567164,18601.990050,17279.042289
8,예술/대중문화,16771.556351,16775.330948,16775.330948,18546.475850,17217.173524
9,인문,16520.914397,16536.031128,16536.031128,18198.832685,16947.952335


### 리뷰 평균

In [30]:
books_main_review = books.groupby(books['main_category']).agg({'aladin_review':'mean', 'kyobo_review': 'mean', 'yes24_review': 'mean', 
                                                        'interpark_review': 'mean'})

books_main_review = books_main_review.withColumn("avg review", (col('avg(aladin_review)') + col('avg(kyobo_review)') + col('avg(yes24_review)') + col('avg(interpark_review)'))/4)
books_main_review.orderBy(col('avg review').desc()).toPandas()  

,main_category,avg(interpark_review),avg(aladin_review),avg(yes24_review),avg(kyobo_review),avg review
0,소설,34.886154,148.603077,98.600000,9.858462,72.986923
1,자기계발,15.029240,47.713450,52.956140,12.116959,31.953947
2,어린이,15.248062,37.134367,46.509044,4.211886,25.775840
3,역사/문화,9.383838,45.181818,37.959596,3.606061,24.032828
4,경제/경영,9.619792,36.041667,42.161458,7.385417,23.802083
5,인문,9.324903,40.498054,34.377432,6.025292,22.556420
6,유아,15.844720,29.605590,40.826087,3.714286,22.497671
7,시/에세이,8.369014,33.500000,24.381690,6.588732,18.209859
8,사회,6.634715,36.787565,23.279793,4.323834,17.756477
9,가정/생활/요리,8.392060,25.146402,31.183623,4.516129,17.309553


In [ ]:
books_main_star.coalesce(1).write.format("com.databricks.spark.csv") \
            .option("header", "true") \
            .save("/home/hadoop/venv/books_main_star.csv")

books_main_price.coalesce(1).write.format("com.databricks.spark.csv") \
            .option("header", "true") \
            .save("/home/hadoop/venv/books_main_price.csv")

books_main_review.coalesce(1).write.format("com.databricks.spark.csv") \
            .option("header", "true") \
            .save("/home/hadoop/venv/books_main_review.csv")

# sub category별 별점, 가격, 리뷰개수 평균

### 별점 평균

In [23]:
books_sub_star = books.groupby(books['sub_category']).agg({'aladin_star':'mean', 'kyobo_star': 'mean', 'yes24_star': 'mean', 
                                                        'interpark_star': 'mean'})

books_sub_star = books_sub_star.withColumn("avg stars", (col('avg(aladin_star)') + col('avg(kyobo_star)') + col('avg(yes24_star)') + col('avg(interpark_star)'))/4)
books_sub_star.limit(10).toPandas()     

,sub_category,avg(aladin_star),avg(kyobo_star),avg(yes24_star),avg(interpark_star),avg stars
0,쉽게 읽는 과학,9.233333,9.566667,9.312121,7.790909,8.975758
1,집/살림,8.693902,9.385366,9.378049,5.606098,8.265854
2,천문/지구과학,8.644156,9.648052,9.329870,6.876623,8.624675
3,한국사,8.273529,9.558824,9.320588,6.672059,8.456250
4,낚시,3.666667,8.866667,9.233333,5.066667,6.708333
5,골프,9.440000,9.260000,9.260000,7.260000,8.805000
6,불교,8.186154,9.509231,9.296923,5.275385,8.066923
7,일본어,8.402222,9.592222,9.393333,6.725556,8.528333
8,서양사,9.100000,9.850000,9.200000,8.300000,9.112500
9,취미기타,8.820513,9.596154,9.564103,6.843590,8.706090


### 가격 평균

In [24]:
books_sub_price = books.groupby(books['sub_category']).agg({'aladin_price':'mean', 'kyobo_price': 'mean', 'yes24_price': 'mean', 
                                                        'interpark_price': 'mean'})

books_sub_price = books_sub_price.withColumn("avg price", (col('avg(aladin_price)') + col('avg(kyobo_price)') + col('avg(yes24_price)') + col('avg(interpark_price)'))/4)
books_sub_price.limit(10).toPandas()    

,sub_category,avg(aladin_price),avg(kyobo_price),avg(interpark_price),avg(yes24_price),avg price
0,쉽게 읽는 과학,18409.090909,18409.090909,18409.090909,20454.545455,18920.454545
1,집/살림,17383.170732,17333.780488,17333.780488,19259.756098,17827.621951
2,천문/지구과학,17099.220779,17091.428571,17091.428571,18955.844156,17559.480519
3,한국사,17940.000000,17888.823529,17888.823529,19835.294118,18388.235294
4,낚시,12900.000000,12900.000000,12900.000000,14333.333333,13258.333333
5,골프,15930.000000,15480.000000,15480.000000,17200.000000,16022.500000
6,불교,13044.923077,13033.384615,13033.384615,14396.923077,13377.153846
7,일본어,14998.444444,14998.444444,14998.444444,16493.222222,15372.138889
8,서양사,26100.000000,26100.000000,26100.000000,29000.000000,26825.000000
9,취미기타,13564.487179,13872.948718,13872.948718,16011.538462,14330.480769


### 리뷰 평균

In [25]:
books_sub_review = books.groupby(books['sub_category']).agg({'aladin_review':'mean', 'kyobo_review': 'mean', 'yes24_review': 'mean', 
                                                        'interpark_review': 'mean'})

books_sub_review = books_sub_review.withColumn("avg review", (col('avg(aladin_review)') + col('avg(kyobo_review)') + col('avg(yes24_review)') + col('avg(interpark_review)'))/4)
books_sub_review.limit(10).toPandas()   

,sub_category,avg(interpark_review),avg(aladin_review),avg(yes24_review),avg(kyobo_review),avg review
0,쉽게 읽는 과학,7.424242,24.151515,27.030303,6.636364,16.310606
1,집/살림,3.646341,12.939024,16.024390,3.914634,9.131098
2,천문/지구과학,5.818182,22.701299,20.831169,4.753247,13.525974
3,한국사,5.705882,25.867647,27.411765,3.558824,15.636029
4,낚시,1.333333,0.888889,2.333333,0.333333,1.222222
5,골프,1.600000,2.800000,6.400000,1.600000,3.100000
6,불교,4.723077,17.400000,9.800000,1.846154,8.442308
7,일본어,4.344444,12.588889,14.522222,2.077778,8.383333
8,서양사,9.500000,17.000000,20.500000,5.000000,13.000000
9,취미기타,5.794872,12.948718,21.820513,5.730769,11.573718


In [ ]:
books_sub_star.coalesce(1).write.format("com.databricks.spark.csv") \
            .option("header", "true") \
            .save("/home/hadoop/venv/books_sub_star.csv")

books_sub_price.coalesce(1).write.format("com.databricks.spark.csv") \
            .option("header", "true") \
            .save("/home/hadoop/venv/books_sub_price.csv")

books_sub_review.coalesce(1).write.format("com.databricks.spark.csv") \
            .option("header", "true") \
            .save("/home/hadoop/venv/books_sub_review.csv")

# 기본 책 정보 csv 저장

In [ ]:
books.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("/home/hadoop/venv/book_info.csv")